# Logistic Regression

#### Introduction
Logistic Regression is a **supervised learning algorithm** used for **classification tasks**. It predicts **probabilities** and is commonly used in **binary classification problems** (e.g., spam detection, disease prediction).

#### How It Works
Instead of predicting a continuous value (like Linear Regression), Logistic Regression predicts the **probability (p) that an instance belongs to a certain class** using the **sigmoid function**:

\[
p = \frac{1}{1 + e^{-(wX + b)}}
\]

Where:
- \( wX + b \) is the **linear equation** (weights and bias),
- \( e \) is Euler’s number (~2.718),
- The result is between **0 and 1**, interpreted as a probability.

#### Decision Boundary
- If \( p \geq 0.5 \), classify as **1** (positive class).
- If \( p < 0.5 \), classify as **0** (negative class).

This forms a **decision boundary** that separates different classes.

#### Cost Function (Log Loss)
Logistic Regression uses **Log Loss (Binary Cross-Entropy)** to measure how well the model predicts:

\[
J(w, b) = -\frac{1}{m} \sum \left[ y \log(\hat{y}) + (1 - y) \log(1 - \hat{y}) \right]
\]

Where:
- \( y \) is the actual class (0 or 1),
- \( \hat{y} \) is the predicted probability,
- \( m \) is the number of samples.

#### Types of Logistic Regression
- **Binary Logistic Regression** → Two classes (e.g., spam or not spam).
- **Multiclass Logistic Regression** → Multiple classes using **One-vs-Rest (OvR)** or **Softmax Regression**.

#### Advantages
- Simple and efficient for classification  
- Interpretable and easy to implement  
- Works well with **linearly separable** data  

#### Limitations
- Struggles with **non-linear** relationships (requires feature engineering or a more complex model).  
- Sensitive to **outliers**.  
- Assumes **independent features** (may require preprocessing).  



In [1]:
import pandas as pd
import numpy as np